In [ ]:
pip install --upgrade --no-cache-dir git+https://github.com/rongardF/tvdatafeed.git

  Cloning https://github.com/rongardF/tvdatafeed.git to /tmp/pip-req-build-1y1m4p9d
  Running command git clone --filter=blob:none --quiet https://github.com/rongardF/tvdatafeed.git /tmp/pip-req-build-1y1m4p9d
  Resolved https://github.com/rongardF/tvdatafeed.git to commit e6f6aaa7de439ac6e454d9b26d2760ded8dc4923
  Preparing metadata (setup.py) ... done
  Created wheel for tvdatafeed: filename=tvdatafeed-2.1.0-py3-none-any.whl size=17533 sha256=f35768c0bd6c715fc627248c6c731f2bf1d9e99f0d992893df13b7fe1a9d4924
  Stored in directory: /tmp/pip-ephem-wheel-cache-fyrz8mb1/wheels/4b/f9/e9/82474a5d9a64702ba7dd5481dee05d267c69bcfa480d382b7c
Successfully built tvdatafeed


In [ ]:
!pip install pandas_ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=45d9138f2d98cca5b3dfd90f3ee917c0be0a82b27aa951e7725ff404c250dc3a
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [ ]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import pandas_ta as ta
import matplotlib.pyplot as plt

def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)

def run_model(stock_symbol):
    tv = TvDatafeed()
    data_daily = tv.get_hist(symbol=stock_symbol, exchange='NSE', interval=Interval.in_daily, n_bars=1500)
    df = pd.DataFrame(data_daily)

    # Calculate RSI
    df['rsi'] = ta.rsi(df['close'], length=7)

    df1_actual = df.reset_index()['close'][:-30].values
    scaler = MinMaxScaler(feature_range=(0,1))
    df1_scaled = scaler.fit_transform(np.array(df1_actual).reshape(-1,1))

    time_step = 100
    X_train, y_train = create_dataset(df1_scaled, time_step)

    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(100,1)))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    model.fit(X_train, y_train, epochs=10, batch_size=64, verbose=1)

    # Initialize variables for tracking trades and returns
    total_returns = 0
    trades = []

    # Loop over 30 days
    for day in range(30):
        # Predict the next day's closing price
        temp_input = df1_scaled[-100:].flatten().tolist()
        x_input = np.array(temp_input).reshape((1, time_step, 1))
        yhat = model.predict(x_input, verbose=0)

        # Get the actual price for the current day
        actual_price_today = df['close'][-(31 - day)]
        price_update=df['close'][-(30 - day)]

        # Append the actual price to df1_actual
        df1_actual = np.append(df1_actual, price_update)

        # Scale the appended array
        df1_scaled = scaler.transform(df1_actual.reshape(-1, 1))

        # Get the predicted price for tomorrow
        predicted_price_tomorrow = scaler.inverse_transform(yhat)[0][0]

        # Buy today if predicted price for tomorrow is higher than today's closing price
        if predicted_price_tomorrow > actual_price_today:
            buy_price_today = actual_price_today
            sell_price_tomorrow = df['close'][-(30 - day)]

            # Calculate daily return
            daily_return = (sell_price_tomorrow - buy_price_today) / buy_price_today * 100
            total_returns += daily_return

            # Record the trade details
            trades.append({
                'day': day + 1,
                'buy_price': buy_price_today,
                'sell_price': sell_price_tomorrow,
                'daily_return': daily_return,
                'win': daily_return > 0
            })
        else:
            # No trade taken if predicted price is not higher
            daily_return = 0

        # Print daily results (optional)
        print(f"Day {day + 1}: Predicted Price for Tomorrow: {predicted_price_tomorrow:.2f}, Actual Price Today: {actual_price_today:.2f}, Daily Return: {daily_return:.2f}%")

    # Calculate overall monthly profit percentage
    overall_returns = total_returns

    return {
        'stock': stock_symbol,
        'overall_returns': overall_returns,
        'trades': trades
    }

def calculate_metrics(results):
    total_trades = 0
    winning_trades = 0
    total_returns = 0

    for result in results:
        total_trades += len(result['trades'])
        winning_trades += sum(1 for trade in result['trades'] if trade['win'])
        total_returns += result['overall_returns']

    win_rate = (winning_trades / total_trades) * 100 if total_trades > 0 else 0
    average_returns = total_returns / len(results) if results else 0

    return {
        'win_rate': win_rate,
        'average_returns': average_returns
    }

# Define list of stock symbols
stock_symbols = ['GRASIM', 'HAL', 'BHEL', 'SIEMENS', 'HDFCBANK', 'ICICIBANK', 'WIPRO', 'RELIANCE', 'INFY', 'BAJAJHLDNG']

# Run the model for each stock and collect results
results = [run_model(stock) for stock in stock_symbols]

# Calculate overall metrics
metrics = calculate_metrics(results)

# Print the overall metrics
print(f"Overall Win Rate: {metrics['win_rate']:.2f}%")
print(f"Average Monthly Returns: {metrics['average_returns']:.2f}%")

# Print the results for each stock
for result in results:
    print(f"\nStock: {result['stock']}")
    print(f"Overall Monthly Profit Percentage: {result['overall_returns']:.2f}%")
    print("Trades:")
    for trade in result['trades']:
        print(f"Day {trade['day']}: Buy Price: {trade['buy_price']:.2f}, Sell Price: {trade['sell_price']:.2f}, Daily Return: {trade['daily_return']:.2f}%")


Epoch 1/10
22/22 [==============================] - 10s 145ms/step - loss: 0.0391
Epoch 2/10
22/22 [==============================] - 3s 145ms/step - loss: 0.0039
Epoch 3/10
22/22 [==============================] - 3s 157ms/step - loss: 0.0013
Epoch 4/10
22/22 [==============================] - 4s 200ms/step - loss: 0.0011
Epoch 5/10
22/22 [==============================] - 3s 147ms/step - loss: 9.9533e-04
Epoch 6/10
22/22 [==============================] - 3s 146ms/step - loss: 9.8747e-04
Epoch 7/10
22/22 [==============================] - 4s 181ms/step - loss: 9.6043e-04
Epoch 8/10
22/22 [==============================] - 4s 173ms/step - loss: 9.3829e-04
Epoch 9/10
22/22 [==============================] - 3s 145ms/step - loss: 9.3545e-04
Epoch 10/10
22/22 [==============================] - 3s 146ms/step - loss: 9.0133e-04
Day 1: Predicted Price for Tomorrow: 2416.69, Actual Price Today: 2345.20, Daily Return: -1.23%
Day 2: Predicted Price for Tomorrow: 2416.43, Actual Price Today: 23

Day 28: Predicted Price for Tomorrow: 2574.61, Actual Price Today: 2761.80, Daily Return: 0.00%
Day 29: Predicted Price for Tomorrow: 2591.91, Actual Price Today: 2802.15, Daily Return: 0.00%
Day 30: Predicted Price for Tomorrow: 2608.70, Actual Price Today: 2801.60, Daily Return: 0.00%
Epoch 1/10
22/22 [==============================] - 10s 151ms/step - loss: 0.0074
Epoch 2/10
22/22 [==============================] - 3s 151ms/step - loss: 4.7480e-04
Epoch 3/10
22/22 [==============================] - 4s 178ms/step - loss: 4.1422e-04
Epoch 4/10
22/22 [==============================] - 4s 170ms/step - loss: 3.5919e-04
Epoch 5/10
22/22 [==============================] - 3s 150ms/step - loss: 3.5089e-04
Epoch 6/10
22/22 [==============================] - 3s 147ms/step - loss: 3.7799e-04
Epoch 7/10
22/22 [==============================] - 4s 204ms/step - loss: 3.6765e-04
Epoch 8/10
22/22 [==============================] - 3s 148ms/step - loss: 3.1053e-04
Epoch 9/10
22/22 [=================

Day 28: Predicted Price for Tomorrow: 5784.69, Actual Price Today: 5539.60, Daily Return: -0.96%
Day 29: Predicted Price for Tomorrow: 5813.83, Actual Price Today: 5486.15, Daily Return: 1.12%
Day 30: Predicted Price for Tomorrow: 5841.33, Actual Price Today: 5547.60, Daily Return: -1.05%
Epoch 1/10
22/22 [==============================] - 10s 146ms/step - loss: 0.0187
Epoch 2/10
22/22 [==============================] - 3s 146ms/step - loss: 0.0011
Epoch 3/10
22/22 [==============================] - 4s 190ms/step - loss: 5.5699e-04
Epoch 4/10
22/22 [==============================] - 4s 157ms/step - loss: 5.9345e-04
Epoch 5/10
22/22 [==============================] - 3s 145ms/step - loss: 5.6358e-04
Epoch 6/10
22/22 [==============================] - 3s 146ms/step - loss: 5.5285e-04
Epoch 7/10
22/22 [==============================] - 5s 208ms/step - loss: 5.0617e-04
Epoch 8/10
22/22 [==============================] - 3s 145ms/step - loss: 5.0571e-04
Epoch 9/10
22/22 [===================

Day 28: Predicted Price for Tomorrow: 313.63, Actual Price Today: 329.50, Daily Return: 0.00%
Day 29: Predicted Price for Tomorrow: 315.45, Actual Price Today: 327.95, Daily Return: 0.00%
Day 30: Predicted Price for Tomorrow: 317.48, Actual Price Today: 332.70, Daily Return: 0.00%
Epoch 1/10
22/22 [==============================] - 10s 211ms/step - loss: 0.0117
Epoch 2/10
22/22 [==============================] - 3s 143ms/step - loss: 9.8738e-04
Epoch 3/10
22/22 [==============================] - 3s 148ms/step - loss: 5.2019e-04
Epoch 4/10
22/22 [==============================] - 3s 146ms/step - loss: 4.1344e-04
Epoch 5/10
22/22 [==============================] - 4s 203ms/step - loss: 3.9031e-04
Epoch 6/10
22/22 [==============================] - 3s 143ms/step - loss: 3.8595e-04
Epoch 7/10
22/22 [==============================] - 3s 145ms/step - loss: 3.9257e-04
Epoch 8/10
22/22 [==============================] - 4s 160ms/step - loss: 3.6860e-04
Epoch 9/10
22/22 [=======================

Day 28: Predicted Price for Tomorrow: 8406.79, Actual Price Today: 7763.90, Daily Return: 1.07%
Day 29: Predicted Price for Tomorrow: 8445.99, Actual Price Today: 7846.65, Daily Return: -0.90%
Day 30: Predicted Price for Tomorrow: 8480.89, Actual Price Today: 7776.00, Daily Return: -2.09%
Epoch 1/10
22/22 [==============================] - 10s 197ms/step - loss: 0.1097
Epoch 2/10
22/22 [==============================] - 3s 150ms/step - loss: 0.0106
Epoch 3/10
22/22 [==============================] - 3s 146ms/step - loss: 0.0056
Epoch 4/10
22/22 [==============================] - 3s 149ms/step - loss: 0.0038
Epoch 5/10
22/22 [==============================] - 5s 214ms/step - loss: 0.0036
Epoch 6/10
22/22 [==============================] - 3s 148ms/step - loss: 0.0034
Epoch 7/10
22/22 [==============================] - 3s 148ms/step - loss: 0.0034
Epoch 8/10
22/22 [==============================] - 4s 162ms/step - loss: 0.0032
Epoch 9/10
22/22 [==============================] - 4s 192ms/

Day 30: Predicted Price for Tomorrow: 1686.56, Actual Price Today: 1621.90, Daily Return: 0.05%
Epoch 1/10
22/22 [==============================] - 8s 149ms/step - loss: 0.0374
Epoch 2/10
22/22 [==============================] - 4s 189ms/step - loss: 0.0037
Epoch 3/10
22/22 [==============================] - 3s 144ms/step - loss: 0.0015
Epoch 4/10
22/22 [==============================] - 3s 144ms/step - loss: 0.0013
Epoch 5/10
22/22 [==============================] - 4s 175ms/step - loss: 0.0012
Epoch 6/10
22/22 [==============================] - 4s 166ms/step - loss: 0.0012
Epoch 7/10
22/22 [==============================] - 3s 144ms/step - loss: 0.0011
Epoch 8/10
22/22 [==============================] - 3s 147ms/step - loss: 0.0011
Epoch 9/10
22/22 [==============================] - 4s 206ms/step - loss: 0.0011
Epoch 10/10
22/22 [==============================] - 3s 145ms/step - loss: 0.0011
Day 1: Predicted Price for Tomorrow: 1130.76, Actual Price Today: 1113.70, Daily Return: 0.66

Day 28: Predicted Price for Tomorrow: 1186.07, Actual Price Today: 1248.25, Daily Return: 0.00%
Day 29: Predicted Price for Tomorrow: 1191.66, Actual Price Today: 1243.20, Daily Return: 0.00%
Day 30: Predicted Price for Tomorrow: 1196.97, Actual Price Today: 1238.45, Daily Return: 0.00%
Epoch 1/10
22/22 [==============================] - 10s 148ms/step - loss: 0.0451
Epoch 2/10
22/22 [==============================] - 3s 147ms/step - loss: 0.0045
Epoch 3/10
22/22 [==============================] - 4s 206ms/step - loss: 0.0019
Epoch 4/10
22/22 [==============================] - 3s 148ms/step - loss: 0.0017
Epoch 5/10
22/22 [==============================] - 3s 147ms/step - loss: 0.0016
Epoch 6/10
22/22 [==============================] - 3s 147ms/step - loss: 0.0016
Epoch 7/10
22/22 [==============================] - 5s 210ms/step - loss: 0.0016
Epoch 8/10
22/22 [==============================] - 3s 147ms/step - loss: 0.0015
Epoch 9/10
22/22 [==============================] - 3s 148ms/st

Day 28: Predicted Price for Tomorrow: 518.45, Actual Price Today: 541.00, Daily Return: 0.00%
Day 29: Predicted Price for Tomorrow: 522.57, Actual Price Today: 535.55, Daily Return: 0.00%
Day 30: Predicted Price for Tomorrow: 526.30, Actual Price Today: 534.10, Daily Return: 0.00%
Epoch 1/10
22/22 [==============================] - 10s 209ms/step - loss: 0.0670
Epoch 2/10
22/22 [==============================] - 3s 145ms/step - loss: 0.0060
Epoch 3/10
22/22 [==============================] - 3s 147ms/step - loss: 0.0020
Epoch 4/10
22/22 [==============================] - 3s 145ms/step - loss: 0.0017
Epoch 5/10
22/22 [==============================] - 5s 211ms/step - loss: 0.0017
Epoch 6/10
22/22 [==============================] - 3s 146ms/step - loss: 0.0016
Epoch 7/10
22/22 [==============================] - 3s 146ms/step - loss: 0.0016
Epoch 8/10
22/22 [==============================] - 3s 158ms/step - loss: 0.0016
Epoch 9/10
22/22 [==============================] - 4s 192ms/step - l

Day 28: Predicted Price for Tomorrow: 3032.48, Actual Price Today: 3180.55, Daily Return: 0.00%
Day 29: Predicted Price for Tomorrow: 3048.86, Actual Price Today: 3168.45, Daily Return: 0.00%
Day 30: Predicted Price for Tomorrow: 3064.65, Actual Price Today: 3161.30, Daily Return: 0.00%
Epoch 1/10
22/22 [==============================] - 9s 144ms/step - loss: 0.0651
Epoch 2/10
22/22 [==============================] - 3s 145ms/step - loss: 0.0052
Epoch 3/10
22/22 [==============================] - 5s 208ms/step - loss: 0.0021
Epoch 4/10
22/22 [==============================] - 3s 145ms/step - loss: 0.0019
Epoch 5/10
22/22 [==============================] - 3s 143ms/step - loss: 0.0018
Epoch 6/10
22/22 [==============================] - 3s 145ms/step - loss: 0.0018
Epoch 7/10
22/22 [==============================] - 5s 207ms/step - loss: 0.0017
Epoch 8/10
22/22 [==============================] - 3s 143ms/step - loss: 0.0016
Epoch 9/10
22/22 [==============================] - 3s 145ms/ste

Day 28: Predicted Price for Tomorrow: 1543.49, Actual Price Today: 1657.15, Daily Return: 0.00%
Day 29: Predicted Price for Tomorrow: 1554.15, Actual Price Today: 1648.25, Daily Return: 0.00%
Day 30: Predicted Price for Tomorrow: 1564.28, Actual Price Today: 1652.70, Daily Return: 0.00%
Epoch 1/10
22/22 [==============================] - 9s 142ms/step - loss: 0.0339
Epoch 2/10
22/22 [==============================] - 3s 143ms/step - loss: 0.0036
Epoch 3/10
22/22 [==============================] - 4s 199ms/step - loss: 0.0016
Epoch 4/10
22/22 [==============================] - 3s 148ms/step - loss: 0.0013
Epoch 5/10
22/22 [==============================] - 3s 142ms/step - loss: 0.0013
Epoch 6/10
22/22 [==============================] - 3s 142ms/step - loss: 0.0012
Epoch 7/10
22/22 [==============================] - 4s 200ms/step - loss: 0.0012
Epoch 8/10
22/22 [==============================] - 3s 142ms/step - loss: 0.0012
Epoch 9/10
22/22 [==============================] - 3s 142ms/ste